In [699]:
%matplotlib inline

In [700]:
import torch
import torchvision
import torchvision.transforms as transforms

**Finding Mean and Std of CIFAR 10 Dataset**

In [701]:
import numpy as np
# load data
from torchvision import datasets
# load the training data
train_data = datasets.CIFAR10('./cifar10_data', train=True, download=True)
# use np.concatenate to stick all the images together to form a 1600000 X 32 X 3 array
x = np.concatenate([np.asarray(train_data[i][0]) for i in range(len(train_data))])
# print(x)
print(x.shape)
# calculate the mean and std along the (0, 1) axes
train_mean = np.mean(x, axis=(0, 1))
train_std = np.std(x, axis=(0, 1))
# the the mean and std
print(train_mean/255, train_std/255)

Files already downloaded and verified
(1600000, 32, 3)
[0.49139968 0.48215841 0.44653091] [0.24703223 0.24348513 0.26158784]


**Data Augmentation**

In [702]:
!pip install albumentations==0.4.6
import albumentations as A
from albumentations.pytorch import ToTensorV2

alb_transform = A.Compose([
    A.HorizontalFlip(p=0.4),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    A.CoarseDropout(max_holes = 1, max_height=4, max_width=4, min_holes = 1, 
                    min_height=1, min_width=1, fill_value=[0.49139968, 0.48215841, 0.44653091]),
    #A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=(0.49139968, 0.48215841, 0.44653091), std=(0.24703223, 0.24348513, 0.26158784)),
    ToTensorV2(),
])

#train_transform = A.Compose(
#    [
#        A.SmallestMaxSize(max_size=160),
#        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
#        A.RandomCrop(height=128, width=128),
#        A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
#        A.RandomBrightnessContrast(p=0.5),
#        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
#        ToTensorV2(),
#    ]
#)

In [703]:
class Cifar10SearchDataset(torchvision.datasets.CIFAR10):
    def __init__(self, root="~/data/cifar10", train=True, download=True, transform=None):
        super().__init__(root=root, train=train, download=download, transform=transform)

    def __getitem__(self, index):
        image, label = self.data[index], self.targets[index]

        if self.transform is not None:
            transformed = self.transform(image=image)
            image = transformed["image"]

        return image, label

In [704]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.49139968, 0.48215841, 0.44653091), (0.229, 0.224, 0.225))])

trainset = Cifar10SearchDataset(transform=alb_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


**Network**

In [705]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1, stride=2, dilation=1)#16
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 112, 3, padding=1, stride=2, dilation=1)#8
        self.bn2 = nn.BatchNorm2d(112)
        self.depth_conv = nn.Conv2d(in_channels=112, out_channels=112, kernel_size=3, groups=112, padding=1, stride=1, dilation=1)#8
        self.point_conv = nn.Conv2d(in_channels=112, out_channels=84, kernel_size=1, stride=1, dilation=1)#8

        self.depthwise_separable_conv = torch.nn.Sequential(self.depth_conv, self.point_conv)
        self.bn3 = nn.BatchNorm2d(84)
        self.conv3 = nn.Conv2d(84, 128, 3, dilation=2)#4
        self.bn4 = nn.BatchNorm2d(128)
        self.pool1 = nn.AvgPool2d(4)#1
        
        #self.conv4 = nn.Conv2d(24, 32, 3)#18 =>1*1*32 =>18*18*n

        self.fc1 = nn.Linear(128, 10)

        self.dropout = nn.Dropout(0.02)

    def forward(self, x):
        x = self.bn1(F.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.bn2(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = self.bn3(F.relu(self.depthwise_separable_conv(x)))
        x = self.dropout(x)
        x = self.bn4(F.relu(self.conv3(x)))
        x = self.dropout(x)
        x= self.pool1(x)

        x = x.view(-1, 128)
        x = self.fc1(x)
        return x


In [706]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = Net().to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           1,792
       BatchNorm2d-2           [-1, 64, 16, 16]             128
           Dropout-3           [-1, 64, 16, 16]               0
            Conv2d-4            [-1, 112, 8, 8]          64,624
       BatchNorm2d-5            [-1, 112, 8, 8]             224
           Dropout-6            [-1, 112, 8, 8]               0
            Conv2d-7            [-1, 112, 8, 8]           1,120
            Conv2d-8            [-1, 112, 8, 8]           1,120
            Conv2d-9             [-1, 84, 8, 8]           9,492
           Conv2d-10             [-1, 84, 8, 8]           9,492
      BatchNorm2d-11             [-1, 84, 8, 8]             168
          Dropout-12             [-1, 84, 8, 8]               0
           Conv2d-13            [-1, 128, 4, 4]          96,896
      BatchNorm2d-14            [-

In [707]:
net = Net().to(device)

In [708]:
SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)


CUDA Available? True


**Train the network**


In [709]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.02, momentum=0.9)
scheduler = StepLR(optimizer, step_size=1, gamma=0.9)

In [722]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.5f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            if( (running_loss/2000) < 0.012):
              scheduler.step()
              scheduler.step()
              print("Lr decr 2")
            elif( (running_loss/2000) < 0.017):
              scheduler.step()
              print("Lr decr 1")
            
            running_loss = 0.0
               

print('Finished Training')

[1,   100] loss: 0.01182
Lr decr 2
[1,   200] loss: 0.01147
Lr decr 2
[1,   300] loss: 0.01176
Lr decr 2
[2,   100] loss: 0.01145
Lr decr 2
[2,   200] loss: 0.01204
Lr decr 1
[2,   300] loss: 0.01176
Lr decr 2
[3,   100] loss: 0.01149
Lr decr 2
[3,   200] loss: 0.01198
Lr decr 2
[3,   300] loss: 0.01183
Lr decr 2
[4,   100] loss: 0.01176
Lr decr 2
[4,   200] loss: 0.01205
Lr decr 1
[4,   300] loss: 0.01167
Lr decr 2
[5,   100] loss: 0.01212
Lr decr 1
[5,   200] loss: 0.01174
Lr decr 2
[5,   300] loss: 0.01178
Lr decr 2
[6,   100] loss: 0.01202
Lr decr 1
[6,   200] loss: 0.01170
Lr decr 2
[6,   300] loss: 0.01229
Lr decr 1
[7,   100] loss: 0.01176
Lr decr 2
[7,   200] loss: 0.01087
Lr decr 2
[7,   300] loss: 0.01174
Lr decr 2
[8,   100] loss: 0.01194
Lr decr 2
[8,   200] loss: 0.01176
Lr decr 2
[8,   300] loss: 0.01140
Lr decr 2
[9,   100] loss: 0.01152
Lr decr 2
[9,   200] loss: 0.01166
Lr decr 2
[9,   300] loss: 0.01197
Lr decr 2
[10,   100] loss: 0.01153
Lr decr 2
[10,   200] loss: 0

**Test the network**



In [727]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device) 
        outputs = net(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %.2f %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 83.59 %


In [712]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 79 %
Accuracy of   car : 92 %
Accuracy of  bird : 60 %
Accuracy of   cat : 47 %
Accuracy of  deer : 77 %
Accuracy of   dog : 75 %
Accuracy of  frog : 86 %
Accuracy of horse : 80 %
Accuracy of  ship : 90 %
Accuracy of truck : 87 %
